In [1]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_data = pd.read_csv('TRAIN.csv')
train_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555.0,1.0,1.0,0,1.0
1,2,5,2015-07-31,6064,625.0,1.0,1.0,0,1.0
2,3,5,2015-07-31,8314,821.0,1.0,1.0,0,1.0
3,4,5,2015-07-31,13995,1498.0,1.0,1.0,0,1.0
4,5,5,2015-07-31,4822,559.0,1.0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...
125509,630,5,2015-04-10,5947,748.0,1.0,0.0,0,1.0
125510,631,5,2015-04-10,5285,640.0,1.0,0.0,0,1.0
125511,632,5,2015-04-10,6430,855.0,1.0,0.0,0,0.0
125512,633,5,2015-04-10,6540,528.0,1.0,0.0,0,1.0


In [4]:
train_data['StateHoliday'].unique()

array(['0', 'a', nan], dtype=object)

In [5]:
train_data['StateHoliday'] = train_data['StateHoliday'].replace([0],'0')

In [6]:
train_data['StateHoliday'].unique()

array(['0', 'a', nan], dtype=object)

In [7]:
train_data['SchoolHoliday'].unique()

array([ 1.,  0., nan])

In [8]:
train_data.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        1
Open             1
Promo            1
StateHoliday     1
SchoolHoliday    1
dtype: int64

In [9]:
train_data = train_data.dropna()


In [10]:
train_data.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [13]:
X = train_data.drop(['Sales','Customers','Date','Store'],axis=1)
Y = train_data['Sales']
Y

0          5263
1          6064
2          8314
3         13995
4          4822
          ...  
125508     5015
125509     5947
125510     5285
125511     6430
125512     6540
Name: Sales, Length: 125513, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size=0.2,random_state=0)

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, ['DayOfWeek','StateHoliday'])
    ])

In [17]:
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression()

In [18]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model_1', model_1)
                             ])

my_pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=True))],
                                                

In [19]:
y_train_pred = my_pipeline.predict(X_train)
y_valid_pred = my_pipeline.predict(X_valid)

In [20]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_train,y_train_pred))

print("The model performance for training set")
print('RMSE is {}'.format(rmse))
print("\n")

rmse = np.sqrt(mean_squared_error(Y_valid,y_valid_pred))

print("The model performance for training set")
print('RMSE is {}'.format(rmse))
print("\n")

The model performance for training set
RMSE is 2785.4526833750674


The model performance for training set
RMSE is 2855.8023868703335




In [24]:
test_data = pd.read_csv('TEST.csv')
test_data = test_data.drop(['Date','Store'],axis=1)
test_data

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,0,0,b,1
1,6,1,0,0,0
2,4,1,1,0,1
3,4,1,0,0,1
4,2,1,0,0,1
...,...,...,...,...,...
305158,2,1,1,0,1
305159,6,1,0,0,0
305160,1,1,0,0,0
305161,6,1,0,0,0


In [25]:
y_test_pred = my_pipeline.predict(test_data)

In [26]:
index = [i for i in range(0,305163)]

In [27]:
output = pd.DataFrame(data=y_test_pred,columns=['Sales'])
output

,Sales
0,4869.127954
1,5974.720499
2,7106.627373
3,7106.627373
4,7401.532307
...,...
305158,7401.532307
305159,5974.720499
305160,8416.362142
305161,5974.720499


In [28]:
output_csv = output.to_csv('output.csv',index=True)

In [29]:
df = pd.read_csv('output.csv')
df.head()

,Unnamed: 0,Sales
0,0,4869.127954
1,1,5974.720499
2,2,7106.627373
3,3,7106.627373
4,4,7401.532307
